In [74]:
using Polyhedra, CDDLib
using DrakeVisualizer
import ColorTypes: RGBA
using PyCall
path_to_my_module = "/home/manuelli/drake-mpc/src/boxrobots/src"
push!(LOAD_PATH, path_to_my_module)
reload("BoxRobots")
import BoxRobots
using DrakeVisualizer
import ColorTypes: RGBA
using Polyhedra: SimpleHRepresentation

br = BoxRobots

BoxRobots

In [75]:
# unshift!(PyVector(pyimport("sys")["path"]), "../")
@pyimport boxatlas
@pyimport boxatlas.boxatlascontroller as boxatlascontroller
@pyimport boxatlas.contactstabilizationutils as csu
CSU = csu.ContactStabilizationUtils

PyObject <class boxatlas.contactstabilizationutils.ContactStabilizationUtils at 0x7fd78efd1808>

In [76]:
# attempt to visualize a state
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();
vis = Visualizer()
delete!(vis)
env, robot = br.make_robot_and_environment()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
br.draw_environment(vis, env)
mass = robot.mass
gravity = robot.gravity
vis_options = br.BoxRobotVisualizerOptions(force_arrow_normalizer=mass*abs(gravity[end]))

BoxRobots.BoxRobotVisualizerOptions
  force_arrow_normalizer: Float64 98.0
  com_radius: Float64 0.1
  contact_point_radius: Float64 0.05
  playback_dt: Float64 0.05


# QPInnerLoopController

In [77]:
miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

t_start = 0.
num_time_steps = 20
dt = 0.05
qp_controller = br.QPInnerLoopController(t_start, soln_data, robot_state, num_time_steps, 
dt)

Optimize a model with 2372 rows, 680 columns and 5958 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 718 rows and 56 columns
Presolve time: 0.01s
Presolved: 1654 rows, 624 columns, 4096 nonzeros
Presolved model has 1140 quadratic objective terms
Variable types: 546 continuous, 78 integer (78 binary)

Root relaxation: objective -1.398097e+03, 2196 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1398.0969    0   74          - -1398.0969      -     -    0s
H    0     0                    -1309.332960 -1398.0969  6.78%     -    0s
     0     2 -1398.0969    0   74 -1309.3330 -1398.0969  6.78%     -    0s
*  379   122              72    -1385.710432 -1391.3730  0.41%   7.1    0

BoxRobots.QPInnerLoopController(0.0,PyObject SolutionData(opt=<boxatlas.contactstabilization.BoxAtlasContactStabilization object at 0x7fd786fb8a50>, states=<utils.trajectory.Trajectory object at 0x7fd7ba43f2d0>, inputs=<utils.trajectory.Trajectory object at 0x7fd7ba433890>, contact_indicator=[<utils.piecewise.Piecewise object at 0x7fd7ba44c350>, <utils.piecewise.Piecewise object at 0x7fd7ba44c890>, <utils.piecewise.Piecewise object at 0x7fd7ba44cdd0>, <utils.piecewise.Piecewise object at 0x7fd7ba433350>], ts=array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ]), solve_time=0.6895830631256104),BoxRobots.BoxRobotState{Float64}(BoxRobots.CentroidalDynamicsState{Float64}([0.0,1.0],[-1.75,0.0]),Dict(:left_hand=>BoxRobots.LimbState{Float64}([-0.35,1.0],[-1.75,0.0],false),:right_foot=>BoxRobots.LimbState{Float64}([0.25,0.0],[-1.75,0.0],true),:left_foot=>BoxRobots.LimbState{Float6

In [78]:
# test contact assignment stuff for inner loop QP
t_current = 0.3
robot_state_test = br.make_robot_state()
robot_state_test.limb_states[:left_hand].in_contact = false
contact_assigment_plan = br.extract_contact_assignment_from_plan(t_current, qp_controller)
contact_assigment = br.compute_contact_assignment(t_current, robot_state_test, qp_controller)

late contact


Dict{Symbol,Array{T,1}} with 4 entries:
  :left_hand  => Any[false,true,true,true,false,false,false,false,false,false,f…
  :right_foot => Any[true,true,true,true,true,true,true,true,true,true,true,tru…
  :left_foot  => Any[true,true,true,true,true,true,true,true,true,true,true,tru…
  :right_hand => Any[false,false,false,false,false,false,false,false,false,fals…

In [80]:
contact_assigment_plan[:left_hand]
contact_assigment[:left_hand]

20-element Array{Any,1}:
 false
  true
  true
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [21]:
miqp_controller = br.MIQPController()
t = 0.0
dt = 0.05
duration = 1.5
@time traj = br.simulate_tspan(robot, miqp_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)

Optimize a model with 2372 rows, 680 columns and 5958 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 718 rows and 56 columns
Presolve time: 0.01s
Presolved: 1654 rows, 624 columns, 4096 nonzeros
Presolved model has 1140 quadratic objective terms
Variable types: 546 continuous, 78 integer (78 binary)

Root relaxation: objective -1.398097e+03, 2196 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1398.0969    0   74          - -1398.0969      -     -    0s
H    0     0                    -1309.332960 -1398.0969  6.78%     -    0s
     0     2 -1398.0969    0   74 -1309.3330 -1398.0969  6.78%     -    0s
*  379   122              72    -1385.710432 -1391.3730  0.41%   7.1    0

In [ ]:
br.playback_trajectory(vis, traj, options=vis_options, playback_speed=0.5)

In [ ]:
miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

# construct the SimpleInnerLoopController
t_start = 0.
inner_loop_controller = br.SimpleInnerLoopController(t_start, soln_data)

t = 0.0
dt = 0.025
duration = 1.0
@time traj = br.simulate_tspan(robot, inner_loop_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)


In [17]:
# test solve speed in python
python_controller = boxatlascontroller.BoxAtlasController()
initial_state = CSU[:make_default_initial_state](python_controller[:robot])
initial_state["vcom"][1] = -1.75
opt = python_controller[:construct_contact_stabilization_optimization](initial_state)
soln_data = opt[:solve]()

Optimize a model with 2372 rows, 680 columns and 5958 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 718 rows and 56 columns
Presolve time: 0.01s
Presolved: 1654 rows, 624 columns, 4096 nonzeros
Presolved model has 1140 quadratic objective terms
Variable types: 546 continuous, 78 integer (78 binary)

Root relaxation: objective -1.398097e+03, 2196 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1398.0969    0   74          - -1398.0969      -     -    0s
H    0     0                    -1309.332960 -1398.0969  6.78%     -    0s
     0     2 -1398.0969    0   74 -1309.3330 -1398.0969  6.78%     -    0s
*  379   122              72    -1385.710432 -1391.3730  0.41%   7.1    0

(PyObject <boxatlas.contactstabilization.BoxAtlasContactStabilization object at 0x7f07eeb35350>,PyObject <utils.trajectory.Trajectory object at 0x7f082379fdd0>,PyObject <utils.trajectory.Trajectory object at 0x7f082374e4d0>,Any[PyObject <utils.piecewise.Piecewise object at 0x7f08237bff50>,PyObject <utils.piecewise.Piecewise object at 0x7f08237474d0>,PyObject <utils.piecewise.Piecewise object at 0x7f0823747a10>,PyObject <utils.piecewise.Piecewise object at 0x7f0823747f50>],[0.0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45  …  0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0],0.6864099502563477)

In [16]:
initial_state["vcom"][1] = -1.75
initial_state[:vcom]
typeof(initial_state)
print(initial_state[:vcom])

[-1.75,0.0]

In [ ]:
# extract solve times from the trajectory of SimulationData
function get_solve_time(data)
    return data.controller_data.solve_time
end

function get_contact_array(data)
    return data.state.limb_states[:left_hand].in_contact 
end

solve_time_array = get_solve_time.(traj.data)
contact_array = get_contact_array.(traj.data)

In [ ]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

In [ ]:
plot(traj.time, solve_time_array, xlabel="time", label="solve time")
plot!(traj.time, contact_array, label="contact indicator")